In [ ]:
import psycopg2
import psycopg2.errors
from pymongo import MongoClient
from cassandra.cluster import Cluster
import mysql.connector
import yaml
import pandas as pd
import os
import time
import sys
from pathlib import Path

with open('docker-compose.yaml', 'r') as file:
    docker_config = yaml.safe_load(file)

postgres_config = docker_config['services']['postgres']
postgres_client = psycopg2.connect(
    host='localhost',
    database=postgres_config['environment']['POSTGRES_DB'],
    user=postgres_config['environment']['POSTG RES_USER'],
    password=postgres_config['environment']['POSTGRES_PASSWORD'],
    port=postgres_config['ports'][0].split(':')[0]
)

mariadb_config = docker_config['services']['mariadb']
mariadb_client = mysql.connector.connect(
    host='localhost',
    database=mariadb_config['environment']['MYSQL_DATABASE'],
    user=mariadb_config['environment']['MYSQL_USER'],
    password=mariadb_config['environment']['MYSQL_PASSWORD'],
    port=mariadb_config['ports'][0].split(':')[0],
    allow_local_infile=True
)

mysql_config = docker_config['services']['mysql']
mysql_client = mysql.connector.connect(
    host='localhost',
    database=mysql_config['environment']['MYSQL_DATABASE'],
    user=mysql_config['environment']['MYSQL_USER'],
    password=mysql_config['environment']['MYSQL_PASSWORD'],
    port=mysql_config['ports'][0].split(':')[0],
    allow_local_infile=True
)

mongo_8_config = docker_config['services']['mongo-8']
mongo_8_config = MongoClient(
    host='localhost',
    port=int(mongo_8_config['ports'][0].split(':')[0])
)

mongo_7_config = docker_config['services']['mongo-7']
mongo_7_config = MongoClient(
    host='localhost',
    port=int(mongo_7_config['ports'][0].split(':')[0])
)

cassandra_config = docker_config['services']['cassandra']
cassandra_client = Cluster(['localhost'], port=cassandra_config['ports'][0].split(':')[0])
cassandra_session = cassandra_client.connect()

try:
    postgres_client.cursor().execute("SELECT 0")
    mariadb_client.cursor(buffered=True).execute("SELECT 1")
    mysql_client.cursor(buffered=True).execute("SELECT 1")
    cassandra_session.execute("SELECT release_version FROM system.local")
    mongo_8_config.admin.command('ping')
    mongo_7_config.admin.command('ping')
except Exception as e:
    print("connection test failed")

In [ ]:
def initialize_postgres_schema(conn, schema):
    if not schema:
        return
    try:
        with conn.cursor() as cur:
            cur.execute(schema)
        conn.commit()
        print("PostgreSQL schema initialization complete.")
    except Exception as e:
        conn.rollback()
        print("Error initializing PostgreSQL schema")


def verify_postgres_tables(conn, expected_tables):
    try:
        with conn.cursor() as cur:
            cur.execute("""
                SELECT table_name
                FROM information_schema.tables
                WHERE table_schema = 'public' AND table_name = ANY(%s);
            """, (expected_tables,))
            existing_tables = {row[0] for row in cur.fetchall()}

        missing_tables = set(expected_tables) - existing_tables
        if not missing_tables:
            print(f"INFO: All PostgreSQL tables exist: {', '.join(expected_tables)}")
            return True
        else:
            print(f"WARNING: Missing PostgreSQL tables: {', '.join(missing_tables)}")
            return False
    except Exception as e:
        print(f"ERROR: Error verifying PostgreSQL tables: {e}")
        return False



In [4]:
with open('db/postgres/schema.sql', 'r') as f:
    sql_schema = f.read()

postgres_tables = ['titles', 'aka_titles', 'ratings', 'people', 'principals', 'episodes', 'title_genres']

initialize_postgres_schema(postgres_client, sql_schema)
verify_postgres_tables(postgres_client, postgres_tables)

PostgreSQL schema initialization complete.
INFO: All PostgreSQL tables exist: titles, aka_titles, ratings, people, principals, episodes, title_genres


True

In [ ]:
def postgres_operation(conn, query, fetch=False):
    """
    Execute a PostgreSQL query and optionally fetch results.
    
    Parameters:
    -----------
    conn : psycopg2 connection
        Database connection
    query : str
        SQL query to execute
    fetch : bool
        Whether to fetch and return results
        
    Returns:
    --------
    Result set if fetch=True, None otherwise
    """
    with conn.cursor() as cur:
        cur.execute(query)
        if fetch:
            return cur.fetchall()
        conn.commit()


def load_insert_data_from_tsv(data_dir, max_rows=100):
    """
    Load data from TSV files for INSERT benchmark operations.
    Loads a subset of rows from each TSV file to use for INSERT benchmarks.
    
    Parameters:
    -----------
    data_dir : str or Path
        Directory containing TSV files
    max_rows : int
        Maximum number of rows to load from each file for INSERT operations
        
    Returns:
    --------
    dict: Dictionary containing test data for each table
    """
    from pathlib import Path
    import csv
    
    data_path = Path(data_dir)
    result = {
        'titles': {},
        'people': {},
        'ratings': {},
        'principals': {},
        'aka_titles': {},
        'episodes': {},
        'title_genres': {}
    }
    
    # Helper to escape SQL strings
    def escape_sql(s):
        if s is None or s == '\\N':
            return None
        return str(s).replace("'", "''")
    
    # Load titles
    titles_file = data_path / 'title.basics.tsv'
    if titles_file.exists():
        with open(titles_file, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f, delimiter='\t')
            for i, row in enumerate(reader):
                if i >= max_rows:
                    break
                tconst = row.get('tconst', '')
                if tconst:
                    result['titles'][tconst] = {
                        'title_type': escape_sql(row.get('titleType', '')),
                        'primary_title': escape_sql(row.get('primaryTitle', '')),
                        'original_title': escape_sql(row.get('originalTitle', '')),
                        'is_adult': row.get('isAdult', '0') == '1',
                        'start_year': row.get('startYear', '') if row.get('startYear') != '\\N' else None,
                        'end_year': row.get('endYear', '') if row.get('endYear') != '\\N' else None,
                        'runtime_minutes': row.get('runtimeMinutes', '') if row.get('runtimeMinutes') != '\\N' else None
                    }
    
    # Load people
    people_file = data_path / 'name.basics.tsv'
    if people_file.exists():
        with open(people_file, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f, delimiter='\t')
            for i, row in enumerate(reader):
                if i >= max_rows:
                    break
                nconst = row.get('nconst', '')
                if nconst:
                    result['people'][nconst] = {
                        'primary_name': escape_sql(row.get('primaryName', '')),
                        'birth_year': row.get('birthYear', '') if row.get('birthYear') != '\\N' else None,
                        'death_year': row.get('deathYear', '') if row.get('deathYear') != '\\N' else None,
                        'primary_profession': escape_sql(row.get('primaryProfession', ''))
                    }
    
    # Load ratings
    ratings_file = data_path / 'title.ratings.tsv'
    if ratings_file.exists():
        with open(ratings_file, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f, delimiter='\t')
            for i, row in enumerate(reader):
                if i >= max_rows:
                    break
                tconst = row.get('tconst', '')
                if tconst:
                    result['ratings'][tconst] = {
                        'average_rating': row.get('averageRating', '0'),
                        'num_votes': row.get('numVotes', '0')
                    }
    
    # Load principals
    principals_file = data_path / 'title.principals.tsv'
    if principals_file.exists():
        with open(principals_file, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f, delimiter='\t')
            for i, row in enumerate(reader):
                if i >= max_rows:
                    break
                key = f"{row.get('tconst', '')}_{row.get('ordering', '')}"
                result['principals'][key] = {
                    'title_id': row.get('tconst', ''),
                    'ordering': row.get('ordering', '0'),
                    'person_id': row.get('nconst', ''),
                    'category': escape_sql(row.get('category', '')),
                    'job': escape_sql(row.get('job', '')) if row.get('job') != '\\N' else None,
                    'characters': escape_sql(row.get('characters', '')) if row.get('characters') != '\\N' else None
                }
    
    # Load aka_titles
    aka_file = data_path / 'title.akas.tsv'
    if aka_file.exists():
        with open(aka_file, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f, delimiter='\t')
            for i, row in enumerate(reader):
                if i >= max_rows:
                    break
                key = f"{row.get('titleId', '')}_{row.get('ordering', '')}"
                result['aka_titles'][key] = {
                    'title_id': row.get('titleId', ''),
                    'ordering': row.get('ordering', '0'),
                    'aka_title': escape_sql(row.get('title', '')),
                    'region': escape_sql(row.get('region', '')) if row.get('region') != '\\N' else None,
                    'language': escape_sql(row.get('language', '')) if row.get('language') != '\\N' else None,
                    'types': escape_sql(row.get('types', '')) if row.get('types') != '\\N' else None,
                    'attributes': escape_sql(row.get('attributes', '')) if row.get('attributes') != '\\N' else None,
                    'is_original_title': row.get('isOriginalTitle', '0') == '1'
                }
    
    # Load episodes
    episodes_file = data_path / 'title.episode.tsv'
    if episodes_file.exists():
        with open(episodes_file, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f, delimiter='\t')
            for i, row in enumerate(reader):
                if i >= max_rows:
                    break
                episode_id = row.get('tconst', '')
                if episode_id:
                    result['episodes'][episode_id] = {
                        'parent_id': row.get('parentTconst', ''),
                        'season_number': row.get('seasonNumber', '') if row.get('seasonNumber') != '\\N' else None,
                        'episode_number': row.get('episodeNumber', '') if row.get('episodeNumber') != '\\N' else None
                    }
    
    # Load title_genres (from title.basics.tsv genres column)
    if titles_file.exists():
        with open(titles_file, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f, delimiter='\t')
            for i, row in enumerate(reader):
                if i >= max_rows:
                    break
                tconst = row.get('tconst', '')
                genres = row.get('genres', '')
                if tconst and genres and genres != '\\N':
                    for genre in genres.split(','):
                        genre = genre.strip()
                        if genre:
                            key = f"{tconst}_{genre}"
                            result['title_genres'][key] = {
                                'title_id': tconst,
                                'genre': genre
                            }
    
    return result


In [ ]:
import time
import psutil
import statistics
from typing import List, Tuple, Callable

class BenchmarkTime:    
    def __init__(self, db_type: str, data_dir: str):
        self.data_dir = data_dir
        self.db_type = db_type
        self.process = psutil.Process()
        self.results = []
    
    def get_results_df(self):
        import pandas as pd
        return pd.DataFrame(self.results)

    def run_benchmark(self, scenarios: List[Tuple[str, List[Tuple[str, Callable]]]], 
                    setup_method: Callable = None, 
                    cleanup_method: Callable = None):
        io_counters_start = psutil.disk_io_counters()
        
        if setup_method:
            setup_method()
            
        for scenario_name, operations in scenarios:
            start_time = time.time()
            cpu_samples = []
            memory_samples = []
            durations = []
            
            for op_name, func in operations:
                cpu_samples.append(self.process.cpu_percent())
                memory_samples.append(self.process.memory_info().rss)
                
                op_start = time.time()
                func()
                op_duration = time.time() - op_start
                durations.append(op_duration)
            
            end_time = time.time()
            total_time = end_time - start_time

            avg_cpu = statistics.mean(cpu_samples) if cpu_samples else 0
            avg_memory = statistics.mean(memory_samples) / (1024 * 1024) if memory_samples else 0
            
            io_counters_end = psutil.disk_io_counters()
            read_mb = (io_counters_end.read_bytes - io_counters_start.read_bytes) / (1024 * 1024)
            write_mb = (io_counters_end.write_bytes - io_counters_start.write_bytes) / (1024 * 1024)
            
            avg_op_time = statistics.mean(durations) if durations else 0
            throughput = len(operations) / total_time if total_time > 0 else 0
            
            scenario_result = {
                'database': self.db_type,
                'data_dir': self.data_dir,
                'scenario': scenario_name,
                'total_time': total_time,
                'operations': len(operations),
                'avg_operation_time': avg_op_time,
                'throughput': throughput,
                'cpu_avg': avg_cpu,
                'memory_avg': avg_memory,
                'disk_read_mb': read_mb,
                'disk_write_mb': write_mb
            }
            self.results.append(scenario_result)
            
            print(f"--- {scenario_name} ({self.db_type}) ---")
            print(f"Total time: {total_time:.4f} seconds")
            print(f"Operations: {len(operations)}")
            print(f"Avg operation time: {avg_op_time:.4f} seconds")
            print(f"Throughput: {throughput:.2f} ops/sec")
            print(f"CPU avg: {avg_cpu:.2f}%")
            print(f"Memory avg: {avg_memory:.2f} MB")
            print(f"Disk read: {read_mb:.2f} MB")
            print(f"Disk write: {write_mb:.2f} MB")
            print()
            
            io_counters_start = io_counters_end
        
        if cleanup_method:
            cleanup_method()
            


In [ ]:
INSERT_TITLE="INSERT Title ?"
INSERT_PERSON="INSERT Person ?"
INSERT_RATING="INSERT Rating ?"
INSERT_PRINCIPAL="INSERT Principal ?"
INSERT_AKA_TITLE="INSERT AKA Title ?"
INSERT_EPISODE="INSERT Episode ?"
INSERT_TITLE_GENRE="INSERT Title Genre ?"

SELECT_TITLE = "SELECT title ?"
SELECT_PERSON = "SELECT person ?"
SELECT_ALL_PEOPLE_IN_TITLE = "SELECT all people that are in the title ?"
SELECT_ALL_EPISODES_FOR_SERIES = "SELECT all episodes for the series ?"
SELECT_ALL_RATINGS_WITH_TITLE_INFO = "SELECT all ratings with title info for all titles in the genre ?"

UPDATE_TITLE_PRIMARY_TITLE = "UPDATE Title ? Primary Title"
UPDATE_ALL_RATINGS_FOR_TITLE = "UPDATE all Ratings for Title ?"
UPDATE_PERSON_PRIMARY_NAME = "UPDATE Person ? Primary Name"
UPDATE_TITLE_START_YEAR = "UPDATE Title ? Start Year"
UPDATE_PERSON_BIRTH_YEAR_FOR_ALL_PEOPLE_IN_TITLE = "UPDATE Person Birth Year for all people that are in the title ?"

DELETE_TITLE = "DELETE Title ?"
DELETE_PERSON = "DELETE Person ?"
DELETE_GENRES_THAT_ARE_IN_THE_TITLE = "DELETE Genres that are in the title ?"
DELETE_PEOPLE_WHO_ARE_IN_TITLE = "DELETE People who are in title ?"

In [ ]:
def load_postgres_data(conn, data_dir):
    """
    Loads data from TSV files into PostgreSQL tables for IMDB dataset.
    """
    import time
    data_path = Path(data_dir)
    
    # Helper function to load TSV using COPY
    def copy_from_tsv(conn, table_name, file_path, columns=None):
        op_time = time.time()
        f_op_time = time.time()
        
        try:
            with conn.cursor() as cur:
                # Open file and use COPY FROM
                with open(file_path, 'r', encoding='utf-8') as f:
                    # Skip header line
                    next(f)
                    f_op_time = time.time()
                    
                    if columns:
                        cur.copy_from(f, table_name, columns=columns, null='\\N', sep='\t')
                    else:
                        cur.copy_from(f, table_name, null='\\N', sep='\t')
                conn.commit()
        except Exception as e:
            conn.rollback()
            print(f"ERROR loading {table_name}: {e}")
            raise
        
        end_time = time.time()
        return op_time, f_op_time, end_time
    
    # Load titles (from title.basics.tsv)
    # Columns: tconst, titleType, primaryTitle, originalTitle, isAdult, startYear, endYear, runtimeMinutes, genres
    titles_file = data_path / 'title.basics.tsv'
    if titles_file.exists():
        op_time, f_op_time, end_time = copy_from_tsv(
            conn, 'titles', titles_file,
            columns=('tconst', 'title_type', 'primary_title', 'original_title', 'is_adult', 'start_year', 'end_year', 'runtime_minutes')
        )
        print(f"INFO: Inserted titles in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")
    
    # Load people (from name.basics.tsv)
    # Columns: nconst, primaryName, birthYear, deathYear, primaryProfession, knownForTitles
    people_file = data_path / 'name.basics.tsv'
    if people_file.exists():
        op_time, f_op_time, end_time = copy_from_tsv(
            conn, 'people', people_file,
            columns=('nconst', 'primary_name', 'birth_year', 'death_year', 'primary_profession')
        )
        print(f"INFO: Inserted people in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")
    
    # Load ratings (from title.ratings.tsv)
    # Columns: tconst, averageRating, numVotes
    ratings_file = data_path / 'title.ratings.tsv'
    if ratings_file.exists():
        op_time, f_op_time, end_time = copy_from_tsv(
            conn, 'ratings', ratings_file,
            columns=('title_id', 'average_rating', 'num_votes')
        )
        print(f"INFO: Inserted ratings in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")
    
    # Load principals (from title.principals.tsv)
    # Columns: tconst, ordering, nconst, category, job, characters
    principals_file = data_path / 'title.principals.tsv'
    if principals_file.exists():
        op_time, f_op_time, end_time = copy_from_tsv(
            conn, 'principals', principals_file,
            columns=('title_id', 'ordering', 'person_id', 'category', 'job', 'characters')
        )
        print(f"INFO: Inserted principals in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")
    
    # Load aka_titles (from title.akas.tsv)
    # Columns: titleId, ordering, title, region, language, types, attributes, isOriginalTitle
    aka_titles_file = data_path / 'title.akas.tsv'
    if aka_titles_file.exists():
        op_time, f_op_time, end_time = copy_from_tsv(
            conn, 'aka_titles', aka_titles_file,
            columns=('title_id', 'ordering', 'aka_title', 'region', 'language', 'types', 'attributes', 'is_original_title')
        )
        print(f"INFO: Inserted aka_titles in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")
    
    # Load episodes (from title.episode.tsv)
    # Columns: tconst, parentTconst, seasonNumber, episodeNumber
    episodes_file = data_path / 'title.episode.tsv'
    if episodes_file.exists():
        op_time, f_op_time, end_time = copy_from_tsv(
            conn, 'episodes', episodes_file,
            columns=('episode_id', 'parent_id', 'season_number', 'episode_number')
        )
        print(f"INFO: Inserted episodes in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")
    
    # Load title_genres (from title.basics.tsv - genres column needs to be split)
    # This requires special handling as genres are comma-separated in the TSV
    if titles_file.exists():
        op_time = time.time()
        try:
            with conn.cursor() as cur:
                with open(titles_file, 'r', encoding='utf-8') as f:
                    # Skip header
                    next(f)
                    f_op_time = time.time()
                    
                    genre_count = 0
                    for line in f:
                        parts = line.strip().split('\t')
                        if len(parts) >= 9:
                            tconst = parts[0]
                            genres = parts[8] if parts[8] != '\\N' else ''
                            if genres:
                                for genre in genres.split(','):
                                    genre = genre.strip()
                                    if genre:
                                        cur.execute(
                                            "INSERT INTO title_genres (title_id, genre) VALUES (%s, %s) ON CONFLICT DO NOTHING",
                                            (tconst, genre)
                                        )
                                        genre_count += 1
                conn.commit()
        except Exception as e:
            conn.rollback()
            print(f"ERROR loading title_genres: {e}")
            raise
        
        end_time = time.time()
        print(f"INFO: Inserted {genre_count} title_genres in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")

In [ ]:
def postgres_benchmark(data_dir, random_numbers_list) -> pd.DataFrame:
    benchmark = BenchmarkTime("postgres", data_dir)
    def setup_for_insert():
        with open('db/postgres/schema.sql', 'r') as f:
            sql_schema = f.read()

        initialize_postgres_schema(postgres_client, sql_schema)
        load_postgres_data(postgres_client, data_dir)

    insert_scenarios = [
        (
            INSERT_TITLE,
            [
                (INSERT_TITLE + str(k), lambda k=k, v=v: postgres_operation(postgres_client, 
                    f"INSERT INTO titles (tconst, title_type, primary_title, original_title, is_adult, start_year, end_year, runtime_minutes) VALUES ('{k}', '{v['title_type']}', '{v['primary_title']}', '{v['original_title']}', {v['is_adult']}, {v['start_year']}, {v['end_year']}, {v['runtime_minutes']})")) 
                for k, v in test_titles.items()
            ]
        ),
        (
            INSERT_PERSON,
            [
                (INSERT_PERSON + str(k), lambda k=k, v=v: postgres_operation(postgres_client,
                    f"INSERT INTO people (nconst, primary_name, birth_year, death_year, primary_profession) VALUES ('{k}', '{v['primary_name']}', {v['birth_year']}, {v['death_year']}, '{v['primary_profession']}')"))
                for k, v in test_people.items()
            ]
        ),
        (
            INSERT_RATING,
            [
                (INSERT_RATING + str(k), lambda k=k, v=v: postgres_operation(postgres_client,
                    f"INSERT INTO ratings (title_id, average_rating, num_votes) VALUES ('{k}', {v['average_rating']}, {v['num_votes']})"))
                for k, v in test_ratings.items()
            ]
        ), 
        (
            INSERT_PRINCIPAL,
            [
                (INSERT_PRINCIPAL + str(k), lambda k=k, v=v: postgres_operation(postgres_client,
                    f"INSERT INTO principals (title_id, ordering, person_id, category, job, characters) VALUES ('{v['title_id']}', {v['ordering']}, '{v['person_id']}', '{v['category']}', " + (f"'{v['job']}'" if v.get('job') else 'NULL') + ", " + (f"'{v['characters']}'" if v.get('characters') else 'NULL') + ")"))
                for k, v in test_principals.items()
            ]
        ),
        (
            INSERT_AKA_TITLE,
            [
                (INSERT_AKA_TITLE + str(k), lambda k=k, v=v: postgres_operation(postgres_client,
                    f"INSERT INTO aka_titles (title_id, ordering, aka_title, region, language, types, attributes, is_original_title) VALUES ('{v['title_id']}', {v['ordering']}, '{v['aka_title']}', " + (f"'{v['region']}'" if v.get('region') else 'NULL') + ", " + (f"'{v['language']}'" if v.get('language') else 'NULL') + ", " + (f"'{v['types']}'" if v.get('types') else 'NULL') + ", " + (f"'{v['attributes']}'" if v.get('attributes') else 'NULL') + f", {v['is_original_title']})"))
                for k, v in test_aka_titles.items()
            ]
        ),
        (
            INSERT_EPISODE,
            [
                (INSERT_EPISODE + str(k), lambda k=k, v=v: postgres_operation(postgres_client,
                    f"INSERT INTO episodes (episode_id, parent_id, season_number, episode_number) VALUES ('{k}', '{v['parent_id']}', {v['season_number']}, {v['episode_number']})"))
                for k, v in test_episodes.items()
            ]
        ),
        (
            INSERT_TITLE_GENRE,
            [
                (INSERT_TITLE_GENRE + str(k), lambda k=k, v=v: postgres_operation(postgres_client,
                    f"INSERT INTO title_genres (title_id, genre) VALUES ('{v['title_id']}', '{v['genre']}')"))
                for k, v in test_title_genres.items()
            ]
        )
    ]

    select_scenarios = [
        (
            SELECT_TITLE, [
                (SELECT_TITLE + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"SELECT * FROM titles WHERE tconst = '{k}'", fetch=True))
                for k in random_numbers_list
            ]
        ),
        (
            SELECT_PERSON, [
                (SELECT_PERSON + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"SELECT * FROM people WHERE nconst = '{k}'", fetch=True))
                for k in random_numbers_list
            ]
        ),
        (
            SELECT_ALL_PEOPLE_IN_TITLE, [
                (SELECT_ALL_PEOPLE_IN_TITLE + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"SELECT * FROM people p JOIN principals pr ON p.nconst = pr.person_id WHERE pr.title_id = '{k}'", fetch=True))
                for k in random_numbers_list
            ]
        ),
        (
            SELECT_ALL_EPISODES_FOR_SERIES, [
                (SELECT_ALL_EPISODES_FOR_SERIES + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"SELECT * FROM episodes e JOIN titles t ON e.episode_id = t.tconst WHERE e.parent_id = '{k}'", fetch=True))
                for k in random_numbers_list
            ] 
        ),
        (
            SELECT_ALL_RATINGS_WITH_TITLE_INFO, [
                (SELECT_ALL_RATINGS_WITH_TITLE_INFO + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"SELECT * FROM ratings r JOIN titles t ON r.title_id = t.tconst JOIN title_genres tg ON t.tconst = tg.title_id WHERE tg.genre = '{k}'", fetch=True))
                for k in random_numbers_list
            ] 
        ),
    ]

    update_scenarios = [
        (
            UPDATE_TITLE_PRIMARY_TITLE, [
                (UPDATE_TITLE_PRIMARY_TITLE + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"UPDATE titles SET primary_title = 'UPDATED' WHERE tconst = '{k}'"))
                for k in random_numbers_list
            ],
        ),
        (
            UPDATE_ALL_RATINGS_FOR_TITLE, [
                (UPDATE_ALL_RATINGS_FOR_TITLE + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"UPDATE ratings SET average_rating = 10.0 WHERE title_id = '{k}'"))
                for k in random_numbers_list
            ],
        ),
        (
            UPDATE_PERSON_PRIMARY_NAME, [
                (UPDATE_PERSON_PRIMARY_NAME + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"UPDATE people SET primary_name = 'UPDATED' WHERE nconst = '{k}'"))
                for k in random_numbers_list
            ],
        ),
        (
            UPDATE_TITLE_START_YEAR, [
                (UPDATE_TITLE_START_YEAR + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"UPDATE titles SET start_year = 2024 WHERE tconst = '{k}'"))
                for k in random_numbers_list
            ],
        ),
        (
            UPDATE_PERSON_BIRTH_YEAR_FOR_ALL_PEOPLE_IN_TITLE, [
                (UPDATE_PERSON_BIRTH_YEAR_FOR_ALL_PEOPLE_IN_TITLE + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"UPDATE people SET birth_year = 2000 WHERE nconst IN (SELECT person_id FROM principals WHERE title_id = '{k}')"))
                for k in random_numbers_list
            ],
        )
    ]

    delete_scenarios = [
        (
            DELETE_TITLE, [
                (DELETE_TITLE + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"DELETE FROM titles WHERE tconst = '{k}'"))
                for k in random_numbers_list
            ]
        ),
        (
            DELETE_PERSON, [
                (DELETE_PERSON + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"DELETE FROM people WHERE nconst = '{k}'"))
                for k in random_numbers_list
            ]
        ),
        (
            DELETE_GENRES_THAT_ARE_IN_THE_TITLE, [
                (DELETE_GENRES_THAT_ARE_IN_THE_TITLE + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"DELETE FROM title_genres WHERE title_id = '{k}'"))
                for k in random_numbers_list
            ]
        ),
        (
            DELETE_PEOPLE_WHO_ARE_IN_TITLE, [
                (DELETE_PEOPLE_WHO_ARE_IN_TITLE + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"DELETE FROM principals WHERE title_id = '{k}'"))
                for k in random_numbers_list
            ]
        )
    ]

    benchmark.run_benchmark(insert_scenarios, setup_method=setup_for_insert)
    benchmark.run_benchmark(select_scenarios)
    benchmark.run_benchmark(update_scenarios)
    benchmark.run_benchmark(delete_scenarios)
    return benchmark.get_results_df()


In [ ]:
DEBUG=False
from random import sample
def run_all_benchmarks(scale, ignore_cassandra=False):
    data_path = './data/scale_' + str(scale)

    # generate_files(output_dir=data_path, scale=adjusted_scale)
    rand_list = sample(range(1, adjusted_scale+1), 10)

    results = []
    postgres_results_df = postgres_benchmark(data_path, rand_list)
    results.append(postgres_results_df)

    merged_df = pd.concat(results, ignore_index=True)
    merged_df.to_csv(f"{data_path}/merged_results.csv", index=False)



In [ ]:
run_all_benchmarks(10000)
run_all_benchmarks(100000)
run_all_benchmarks(1000000)

NameError: name 'generate_files' is not defined